First import all the needed modules:

In [ ]:
from configparser import ConfigParser
from sshtunnel import SSHTunnelForwarder
import psycopg2
import pandas as pd

Below, the different PostGreSQL queries can be written and stored in a variable:

In [ ]:
SaleInfoQ = "SELECT sale_order_line.id, DATE(sale_order.confirmation_date) AS confirmation_date, sale_order_line.order_id, sale_order.name, product_template.default_code, product_uom_qty, price_unit, price_subtotal FROM sale_order_line JOIN product_product ON sale_order_line.product_id = product_product.id JOIN product_template ON product_product.product_tmpl_id = product_template.id JOIN sale_order ON sale_order_line.order_id = sale_order.id WHERE sale_order.confirmation_date > '2021-01-01 00:00:00';"


1. Read the credentials from the database connection file
2. Setup the ssh tunnel to the database server
3. Create connection to the database
4. Run the queries
5. Close database connection
6. Close tunnel

In [ ]:
cred = ConfigParser(allow_no_value=True)
cred.read('database.ini')

tunnel = SSHTunnelForwarder(
         (cred['ssh']['Host'], int(cred['ssh']['Port'])),
         ssh_username=cred['ssh']['User'],
         ssh_password=cred['ssh']['Password'],
         remote_bind_address=('localhost', int(cred['database']['Port'])),
         local_bind_address=('localhost', int(cred['database']['Port']))
         )
tunnel.start()

conn = psycopg2.connect(
    host="localhost",
    database=cred['database']['Name'],
    user=cred['database']['User'],
    password=cred['database']['Password']
    )
# ADD THE QUERIES THAT NEED TO SEND TO THE DATABASE HERE

SaleInfo = pd.io.sql.read_sql_query(SaleInfoQ, conn)

# CLOSING CONNECTION AND TUNNEL AFTER RUNNING THE QUERIES
conn.close()
if conn.closed:
    print("Connection closed")
tunnel.stop()
